In [21]:
import pandas as pd
import numpy as np

In [22]:
def read_training():
    training_item = {}
    with open('trainItem2.txt', 'r') as f:
        lines = f.readlines()
        count = 0
        while(count < len(lines)):
            ID = []
            first_line = lines[count]
            userID, ite = lines[count].split('|')
            ite = int(ite)
            for i in range(ite):
                    ID.append(lines[count+i+1].split())
            count += ite+1
            training_item[userID] = dict(ID)
    #print(training_item['199810'])
    return training_item

In [23]:
def read_testing():
    testing_item = {}
    with open('testItem2.txt', 'r') as g:
        lines = g.readlines()
        count = 0
        while(count < len(lines)):
            ID = []
            first_line = lines[count]
            userID, ite = lines[count].split('|')
            ite = int(ite)
            for i in range(ite):
                    ID.append(lines[count+i+1].strip('\n'))
            #lists = [0 for n in range(len(ID))]
            #testing_item[userID] = dict(zip(ID,lists))
            testing_item[userID] = ID
            count += ite+1
    #print(testing_item['199810'])
    return testing_item

In [24]:
def read_track():
    track_data = {}
    with open('trackData2.txt', 'r') as h:
        lines = h.readlines()
        count = 0
        while(count < len(lines)):
            ID = []
            items= []
            items = lines[count].strip('\n').split('|')
            track_data[items.pop(0)] = items
            count += 1
    #print(track_data['1'])
    return track_data

In [25]:
def score():
    for user in testing_item:
        scores = [0 for n in range(6)]
        testing_item[user] = dict(zip(testing_item[user], scores))
        for trackID in testing_item[user]:
            for each in track_data[trackID]:
                if(each in training_item[user]):
                    testing_item[user][trackID] += int(training_item[user][each])
        #print(testing_item[user])

In [36]:
csvlist=['prediction.csv','mf_sum.csv','pre_cvgbt2.csv','pre_dt2.csv','pre_gbt2.csv','pre_lr2.csv','pre_rf2.csv']

In [37]:
P = [0.85269,0.78474,0.87356,0.66551,0.86500,0.79054,0.85755]
stx = []
for i in range(7):
    stx.append(120000*(2*P[i]-1))
sTx = np.array([stx]).T
s = []
for a in range(120000):
    s.append([])

In [33]:
count=0
for i in csvlist:
    file=pd.read_csv(i)
    l=file['Predictor'].tolist()
    for j in range(len(l)):
        s[j].append(l[j])
    count=count+1
S=np.array(s)

In [29]:
for i in range(len(S)):
    for j in range(7):
        S[i][j] = 2*S[i][j]-1

In [30]:
S

array([[-1, -1, -1, ...,  1, -1, -1],
       [ 1, -1, -1, ..., -1, -1, -1],
       [-1, -1,  1, ..., -1, -1,  1],
       ...,
       [ 1,  1,  1, ...,  1,  1,  1],
       [-1, -1, -1, ..., -1, -1, -1],
       [ 1,  1,  1, ...,  1,  1,  1]])

In [14]:
tra = np.linalg.inv(S.transpose().dot(S))

In [15]:
S_temp = np.dot(S,tra)

In [16]:
S_ensemble = np.dot(S_temp,sTx)

In [17]:
print(S_temp)

[[-2.26709305e-05 -1.00716260e-05 -1.01381455e-04 ...  1.69591678e-04
  -1.49108827e-05 -5.91589510e-05]
 [ 3.34413901e-05 -5.98638139e-06 -1.16339166e-05 ... -9.64657124e-06
  -8.98620310e-06 -9.55568999e-06]
 [-1.70011783e-05 -8.37128520e-06  6.18858255e-05 ... -7.57242361e-05
  -1.14417326e-05  3.98109515e-05]
 ...
 [ 3.45235235e-06  2.70299354e-06  6.97443587e-07 ...  9.63210220e-07
   1.05550463e-06  9.21974919e-08]
 [-3.45235235e-06 -2.70299354e-06 -6.97443587e-07 ... -9.63210220e-07
  -1.05550463e-06 -9.21974919e-08]
 [ 3.45235235e-06  2.70299354e-06  6.97443587e-07 ...  9.63210220e-07
   1.05550463e-06  9.21974919e-08]]


In [18]:
for counter in range(0,len(S_ensemble),6):
    location=[0,0,0]
    for index in range(0,6):
        if S_ensemble[counter+index]>S_ensemble[location[0]]:
            location[2]=location[1]
            location[1]=location[0]
            location[0]=counter+index
        elif S_ensemble[counter+index]>S_ensemble[location[1]]:
            location[2]=location[1]
            location[1]=counter+index
        elif S_ensemble[counter+index]>S_ensemble[location[2]]:
            location[2]=counter+index
        else:
            continue
    for i in range(counter,counter+6):
        if i in location:
            S_ensemble[i]=1
        else:
            S_ensemble[i]=0

In [19]:
final = S_ensemble.transpose()[0].tolist()

In [20]:
training_item = read_training()
testing_item = read_testing()
track_data = read_track()
score()
a = []
for every_user in testing_item:
    for every_trackID in testing_item[every_user]:
        a.append(every_user+'_'+every_trackID)
dataframe = pd.DataFrame({'TrackID':a, 'Predictor':final})
dataframe.to_csv("prediction_last.csv",index=False)
print("Succeed!!!")

Succeed!!!
